In [2]:
import pandas as pd
import nltk
import re
import numpy as np
# Used for confusion model which is to tell you how accurate your model is
from sklearn import metrics
# Removing filler words
from nltk.corpus import stopwords
# Simplifying words to base value
from nltk.stem import WordNetLemmatizer
# Train test split (66% train - 33% test)
from sklearn.model_selection import train_test_split
# Used for making str into int (kind of)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection  import KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer


# Logistic Regression model computes probability of an outcome based on an input variable(s)
from sklearn.linear_model import LogisticRegression
df1 = pd.read_csv(r'C:\\Users\keena\\Desktop\\ML & AI\\militaria_products_data.csv',delimiter='|')

df = df1.loc[(df1['conflict'].notnull())&(df1['nation'].notnull())&(df1['item_type'].notnull())]
# df1.dropna(subset=['conflict','nation','item_type']) I think this does the same thing but in place.

df['text'] = df['title'] + ' ' + df['description']
data = df[['text','conflict','nation','item_type']]



C:\Users\keena\AppData\Local\Temp\ipykernel_23168\669913939.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['title'] + ' ' + df['description']


In [3]:
print(data['text'].isna().sum())
print(data['item_type'].isna().sum())
print(df1.isna().sum().sort_values())

0
0
url                 0
price               0
available           0
date                0
site                0
id                  0
currency            0
description         2
title              53
nation           6409
conflict        15289
extracted_id    32921
item_type       33935
date_sold       49740
grade           73131
dtype: int64


In [4]:
text = list(data['text'].dropna())
# Preprocess the text / data
lemmatizer = WordNetLemmatizer()
# Corpus is used when having a list of lists used for analysis
corpus = []
#Condensing the text to only key components
for i in range(len(text)):
    # Replace any non letters with ' ' with text[i] being the current iteration
    r = re.sub('[^a-zA-Z]',' ',text[i])
    # Make everything lowercase
    r = r.lower()
    # Split every word into an item in a list
    r = r.split()
    # Remove all stopwords / filler words
    r = [word for word in r if word not in stopwords.words('english')]
    # Reduce words to base word
    r = [lemmatizer.lemmatize(word) for word in r]
    # Reconnect all the words in a list back into one list item
    r = ' '.join(r)
    # Put this singular list with all the cleaned words into 'corpus'
    corpus.append(r)
data['text'] = corpus
data['text'].head()

C:\Users\keena\AppData\Local\Temp\ipykernel_23168\1887581878.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = corpus


2     german wwi officer shoulder board ir original ...
9     insignia cloth british british wwii army shoul...
18    u wwii army nurse corp blue belted jacket nurs...
31    german wwi dress bayonet frog original era man...
40    insignia metal u wwii usa captain sweetheart p...
Name: text, dtype: object

In [5]:
# Independent Variable, what you have.
X = data['text']
# Dependent Variable, what you want.
y  = data['item_type']
y2 = data['conflict']
y3 = data['nation']

# Split the data to train / test
X_train, X_test, y_train, y_test   = train_test_split(X, y, test_size=0.33, random_state=111)
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.33, random_state=111)
X_train, X_test, y3_train, y3_test = train_test_split(X, y3, test_size=0.33, random_state=111)



In [6]:
cv = CountVectorizer()
lr = LogisticRegression()
lr2 = LogisticRegression()
lr3 = LogisticRegression()

vectorizer = TfidfVectorizer()

X_train_cv = cv.fit_transform(X_train)

lr.fit(X_train_cv,y_train)
lr2.fit(X_train_cv,y2_train)
lr3.fit(X_train_cv,y3_train)

# Convert all the data sets into numeric values for ML to read.
X_test_cv = cv.transform(X_test)

X_train_cv.shape

predictions = lr.predict(X_test_cv)
predictions2 = lr2.predict(X_test_cv)
predictions3 = lr3.predict(X_test_cv)


predictions

array(['EDGED_WEAPONS', 'UNIFORM', 'INSIGNIA', ..., 'HELMET',
       'EDGED_WEAPONS', 'MEDALS_AWARDS'], dtype=object)

In [7]:
df = pd.DataFrame(metrics.confusion_matrix(y_test,predictions))

In [8]:
# This prints out an easy to understand report of how good the model is.
print (classification_report(y_test, predictions))

print (classification_report(y2_test, predictions2))

print (classification_report(y3_test, predictions3))

# This is to see how accurate our prediction model is.
kf = KFold(n_splits=8, shuffle=True, random_state=5)
cv_scores = cross_val_score(lr, X_test_cv, y_test, cv=kf)

# Print the mean
print(np.mean(cv_scores))

# Print the standard deviation
print(np.std(cv_scores))

# Print the 95% confidence interval
print(np.quantile(cv_scores, [0.025, 0.975]))

               precision    recall  f1-score   support

          ART       0.97      0.85      0.91        73
BELTS_BUCKLES       0.98      0.98      0.98       874
EDGED_WEAPONS       0.98      0.98      0.98      1633
   FIELD_GEAR       0.98      0.97      0.98      1010
         FLAG       0.97      0.94      0.95       476
     HEADGEAR       0.98      0.97      0.98       678
       HELMET       0.97      0.98      0.98       846
     INSIGNIA       0.96      0.98      0.97      1865
MEDALS_AWARDS       0.98      0.98      0.98      1699
    PAPERWORK       0.95      0.93      0.94       196
     POSTCARD       0.98      1.00      0.99       260
       TINNIE       1.00      0.99      0.99       210
      UNIFORM       0.96      0.96      0.96       862

     accuracy                           0.97     10682
    macro avg       0.97      0.96      0.97     10682
 weighted avg       0.97      0.97      0.97     10682

                precision    recall  f1-score   support

     

C:\Users\keena\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\keena\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\keena\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

                precision    recall  f1-score   support

     AUSTRALIA       1.00      0.60      0.75        15
       AUSTRIA       0.94      0.91      0.92        64
       BELGIUM       1.00      0.78      0.88        27
      BULGARIA       1.00      0.64      0.78        11
        CANADA       1.00      0.63      0.77        19
         CHINA       0.60      0.38      0.46         8
CZECHOSLOVAKIA       1.00      0.36      0.53        14
       DENMARK       0.00      0.00      0.00         1
       FINLAND       1.00      0.71      0.83         7
        FRANCE       0.93      0.91      0.92       106
       GERMANY       0.99      0.99      0.99      6866
         ITALY       0.92      0.90      0.91        84
         JAPAN       0.99      0.98      0.98       590
   NETHERLANDS       0.71      0.45      0.56        11
        POLAND       0.85      0.69      0.76        16
        POLISH       0.00      0.00      0.00         1
       ROMANIA       1.00      0.33      0.50  

In [9]:

pdData = {'testText': ["""


WWI GERMAN ARMY BUCKLE
WWI German Army Buckle-This is an early steel combat buckle that retains the majority of its original olive-green finish. There is no visible wear. The reverse features a "U" shaped catch with steel roll bar and prongs. This is an excellent example of an early Heer buckle! 


"""]}

tdf = pd.DataFrame(pdData)

text = list(tdf['testText'].dropna())
# Preprocess the text / data
lemmatizer = WordNetLemmatizer()
# Corpus is used when having a list of lists used for analysis
corpus2 = []
#Condensing the text to only key components
for i in range(len(text)):
    # Replace any non letters with ' ' with text[i] being the current iteration
    r = re.sub('[^a-zA-Z]',' ',text[i])
    # Make everything lowercase
    r = r.lower()
    # Split every word into an item in a list
    r = r.split()
    # Remove all stopwords / filler words
    r = [word for word in r if word not in stopwords.words('english')]
    # Reduce words to base word
    r = [lemmatizer.lemmatize(word) for word in r]
    # Reconnect all the words in a list back into one list item
    r = ' '.join(r)
    # Put this singular list with all the cleaned words into 'corpus'
    corpus2.append(r)
tdf['testText'] = corpus2
print(tdf['testText'].head())

tdfTest = tdf['testText']

0    wwi german army buckle wwi german army buckle ...
Name: testText, dtype: object


In [10]:

tdfTest_cv = cv.transform(tdfTest)


itemTypePrediction = lr.predict(tdfTest_cv)
conflictPrediction = lr2.predict(tdfTest_cv)
nationPrediction   = lr3.predict(tdfTest_cv)

print(f'ITEM_TYPE PREDICTION :{itemTypePrediction}')
print(f'CONFLICT PREDICTION  :{conflictPrediction}')
print(f'NATION PREDICTION    :{nationPrediction}')


ITEM_TYPE PREDICTION :['BELTS_BUCKLES']
CONFLICT PREDICTION  :['WW1']
NATION PREDICTION    :['GERMANY']
